# Talkativ: LLM 프롬프트 엔지니어링 & API 호출

## 목차
1. **아바타 대화 생성** - 시스템 프롬프트 설계
2. **실시간 피드백 생성** - 공손성 피드백
3. **토픽 라벨링** - AI-Hub 데이터 분류
4. **대화 가이드 생성** - 관계 기반 가이드
5. **표현 추천** - 더 나은 표현 제안

각 섹션에서 다루는 내용:
- **전체 프롬프트** (System + User)
- **입력 예시**
- **출력 예시**
- **의도 달성 수준**
- **개선책**

## 데모용 CLOVA Studio (HyperCLOVA X) API 사용 (OpenAI 호환 API)

이 노트북은 **OpenAI Python SDK(`openai`)를 그대로 사용**하되,
`base_url`을 CLOVA Studio의 OpenAI 호환 엔드포인트로 바꿔 **HyperCLOVA X 모델을 호출**하는 방식으로 데모를 구성합니다.

아래 셀에서는 다음을 설정합니다.
- `CLOVASTUDIO_API_KEY` 환경변수에서 API 키를 읽기
- `base_url="https://clovastudio.stream.ntruss.com/v1/openai"` 지정
- `MODEL`에 CLOVA Studio 콘솔에서 제공되는 모델명을 입력
- `demo_chat_once()`로 1회 호출을 빠르게 테스트


- 본 데모는 **NAVER CLOVA Studio(HyperCLOVA X)**를 대상으로, Python requests 기반의 HTTP REST 호출로 LLM 응답을 생성한다.

- CLOVA Studio는 OpenAI-compatible API를 제공하므로, 요청 스키마를 POST /v1/openai/chat/completions의 **표준 JSON 형식(messages 기반)**으로 구성한다.

- 모델 호출은 base_url에 해당하는 CLOVA Studio 엔드포인트로 라우팅되며, OpenAI 인프라를 사용하지 않는다.

- 인증은 Authorization: Bearer <API_KEY> 헤더로 수행하고, 키는 환경변수로 관리하여 소스 코드 내 하드코딩을 방지한다.

- 생성 파라미터(temperature, max_tokens)를 통해 출력 다양성과 응답 길이 상한을 제어하며, 결과 텍스트는 응답의 choices[0].message.content에서 추출한다.

In [2]:
import os
os.environ["CLOVASTUDIO_API_KEY"] = "nv-dab6dde4743d4e9694564024dba1b2f1TC9Y"

In [3]:
!pip install -q openai

In [4]:
import os
import json
from typing import List, Dict, Optional
from dataclasses import dataclass, field
from datetime import datetime
from openai import OpenAI

# ============================================================
# CLOVA Studio (HyperCLOVA X) - OpenAI 호환 API 데모 설정
# ============================================================
# 이 코드는 OpenAI Python SDK(OpenAI 클래스)를 그대로 사용하면서,
# CLOVA Studio의 OpenAI 호환 엔드포인트(base_url)로 요청을 보내는 예시입니다.
#
# 1) API 키 관리(권장)
#    - 코드에 키를 직접 적지 말고 환경변수로 관리하세요.
#    - Colab/로컬에서 실행 전:
#        export CLOVASTUDIO_API_KEY="nv-dab6dde4743d4e9694564024dba1b2f1TC9Y"
#
# 2) 엔드포인트(base_url)
#    - CLOVA Studio의 OpenAI 호환 API 엔드포인트를 지정합니다.
#      base_url="https://clovastudio.stream.ntruss.com/v1/openai"
#
# 3) 모델명(MODEL)
#    - CLOVA Studio 콘솔에서 제공되는 모델명을 그대로 입력합니다.
#    - 예: "HCX-005"
#
# 4) 호출 방식
#    - OpenAI SDK의 chat.completions.create()를 사용합니다.
#    - messages는 [{"role":"system"...}, {"role":"user"...}] 형식입니다.
# ============================================================

CLOVASTUDIO_API_KEY = os.getenv("CLOVASTUDIO_API_KEY")
if not CLOVASTUDIO_API_KEY:
    raise RuntimeError(
        "환경변수 CLOVASTUDIO_API_KEY 가 설정되지 않았습니다. "
        "예) export CLOVASTUDIO_API_KEY='발급받은_키'"
    )

client = OpenAI(
    api_key=CLOVASTUDIO_API_KEY,
    base_url="https://clovastudio.stream.ntruss.com/v1/openai",
)

MODEL = "HCX-005"


def demo_chat_once(
    user_text: str,
    system_text: str = "너는 Talkativ 데모용 한국어 회화 튜터야. 간결하고 친절하게 답해.",
    temperature: float = 0.7,
    max_tokens: int = 256,
) -> str:
    """
    1회성 데모 호출 함수.

    Parameters
    ----------
    user_text : str
        사용자의 입력 문장
    system_text : str
        시스템 프롬프트(역할/규칙). 모델의 말투와 정책을 고정하기 위해 사용합니다.
    temperature : float
        창의성/다양성 조절 값. 낮을수록 더 결정적이고, 높을수록 더 다양해집니다.
    max_tokens : int
        생성되는 답변의 최대 토큰 수. 너무 크게 잡으면 비용/지연이 늘어날 수 있습니다.

    Returns
    -------
    str
        모델이 생성한 assistant 메시지(텍스트)
    """
    completion = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_text},
            {"role": "user", "content": user_text},
        ],
        temperature=temperature,
        max_tokens=max_tokens,
    )

    # OpenAI 호환 응답 구조: choices[0].message.content 에 최종 텍스트가 들어갑니다.
    return completion.choices[0].message.content


print("Setup complete! CLOVA Studio client is ready.")


Setup complete! CLOVA Studio client is ready.


In [5]:
# 빠른 동작 테스트
# 아래 문장을 바꿔가며 연결/응답이 정상인지 확인하세요.
print(demo_chat_once("한국어로 토커티브 코칭을 하기 전에 자기소개 한 문장만 만들어줘."))

안녕하세요! 저는 CLOVA X입니다. 자연어 이해와 생성 능력을 바탕으로 사용자님께 도움이 되는 정보를 제공하고 다양한 요청을 수행하기 위해 항상 최선을 다하겠습니다. 잘 부탁드립니다!


### 코드 설명(상세)

####  모델명(MODEL)
- `MODEL`은 **CLOVA Studio 콘솔에서 제공되는 모델명**을 정확히 입력해야 합니다.
- 모델명이 다르면 아래와 같은 문제가 발생할 수 있습니다.
  - 요청은 성공했지만 모델을 찾지 못해 오류 반환
  - 권한이 없는 모델을 호출하여 인증/권한 오류 발생

####  messages 구조와 시스템 프롬프트의 역할
- `messages`는 대화 히스토리를 담는 배열입니다.
  - `"system"`: 규칙/역할/말투/금지사항 등(시스템 프롬프트)
  - `"user"`: 사용자의 실제 입력
- Talkativ처럼 **아바타의 캐릭터와 정책을 일관되게 유지**하려면 `system_text`가 특히 중요합니다.

####  생성 파라미터(temperature, max_tokens)
- `temperature`: 답변의 다양성 조절(낮을수록 안정적, 높을수록 창의적)
- `max_tokens`: 답변 길이 상한
  - 너무 작으면 답이 잘릴 수 있고,
  - 너무 크면 지연/비용이 증가할 수 있습니다.


In [6]:
# 프로필 데이터 구조
@dataclass
class PersonProfile:
    name: str
    age: int
    role: str
    nationality: str = "Korean"
    korean_level: str = "native"
    speaking_anxiety: str = "medium"
    personality_tags: List[str] = field(default_factory=list)
    personality_text: str = ""
    topic_tags: List[str] = field(default_factory=list)
    avoid_topics: List[str] = field(default_factory=list)
    style_description: str = ""
    background: str = ""

# 샘플 프로필
user_jimin = PersonProfile(
    name="지민",
    age=22,
    role="student",
    nationality="American",
    korean_level="intermediate",
    speaking_anxiety="high",
    personality_tags=["introverted", "careful"],
    personality_text="조용하고 신중한 성격",
    topic_tags=["kpop", "cafe_food", "campus_life"],
    avoid_topics=["politics", "religion"]
)

avatar_professor = PersonProfile(
    name="김 교수님",
    age=52,
    role="professor",
    personality_text="학문적이고 진지하지만 학생을 배려하는 교수",
    topic_tags=["class_study", "career_future"],
    style_description="격식체 존댓말, 학생에게도 존중하는 어조",
    background="전공 교수님, 지민의 지도교수"
)

avatar_senior = PersonProfile(
    name="민수 선배",
    age=25,
    role="senior",
    personality_text="후배를 잘 챙기고 친근한 성격",
    topic_tags=["campus_life", "kpop", "part_time_job"],
    style_description="친근한 말투, 격려하는 표현 사용",
    background="같은 학과 3학년 선배"
)

print(" Sample profiles created!")

 Sample profiles created!


---
# 1. 아바타 대화 생성 (Avatar Chat Generation)

## 목적
아바타가 사용자와 자연스러운 한국어 대화를 수행하도록 함

## 핵심 요구사항
- 역할(role)에 맞는 말투 사용
- 관계에 적절한 존댓말 수준
- 사용자의 한국어 수준 고려
- 회피 토픽 언급 금지

In [7]:
# ============================================================
# 1. 아바타 대화 생성 - 시스템 프롬프트
# ============================================================

def generate_avatar_system_prompt(
    user: PersonProfile,
    avatar: PersonProfile,
    recommended_formality: str,
    safe_topics: List[str]
) -> str:
    """
    아바타 대화용 시스템 프롬프트 생성

    이 프롬프트는 HyperCLOVA X 또는 GPT에 전달되어
    아바타가 사용자와 대화할 때의 행동을 정의합니다.
    """

    # 말투 가이드 생성
    formality_guides = {
        "very_polite": "격식체 존댓말을 사용합니다 (–습니다, –습니까). 정중하고 공식적인 어조를 유지합니다.",
        "polite": "일반 존댓말을 사용합니다 (–요, –세요). 친근하지만 예의 바른 어조입니다.",
        "mixed": "기본적으로 존댓말을 사용하되, 가끔 친근한 반말을 섞습니다.",
        "informal": "편한 반말을 사용합니다. 친구처럼 편하게 대화합니다."
    }

    formality_guide = formality_guides.get(recommended_formality, formality_guides["polite"])
    safe_topics_str = ', '.join(safe_topics) if safe_topics else '일상적인 대화'
    avoid_topics_str = ', '.join(set(user.avoid_topics) | set(avatar.avoid_topics))

    # ========================================
    # 전체 시스템 프롬프트
    # ========================================
    system_prompt = f"""당신은 '{avatar.name}'입니다. 한국어로 대화하세요.

═══════════════════════════════════════════════════════════════
[아바타 프로필]
═══════════════════════════════════════════════════════════════
- 이름: {avatar.name}
- 나이: {avatar.age}세
- 역할: {avatar.role}
- 성격: {avatar.personality_text}
- 관심사: {', '.join(avatar.topic_tags)}
- 배경: {avatar.background}

═══════════════════════════════════════════════════════════════
[사용자 프로필]
═══════════════════════════════════════════════════════════════
- 이름: {user.name}
- 나이: {user.age}세
- 역할: {user.role}
- 한국어 수준: {user.korean_level} (외국인 학습자)
- 국적: {user.nationality}
- 말하기 불안도: {user.speaking_anxiety}

═══════════════════════════════════════════════════════════════
[말투 가이드]
═══════════════════════════════════════════════════════════════
권장 말투 수준: {recommended_formality}
{formality_guide}
{avatar.style_description}

═══════════════════════════════════════════════════════════════
[대화 주제]
═══════════════════════════════════════════════════════════════
공통 관심사 (추천): {safe_topics_str}
회피 토픽 (절대 먼저 꺼내지 말 것): {avoid_topics_str}

═══════════════════════════════════════════════════════════════
[지시사항]
═══════════════════════════════════════════════════════════════
1. {avatar.name}의 역할과 성격에 맞게 자연스럽게 대화하세요.
2. 한국어로만 응답하세요.
3. 응답은 1-3문장으로 짧게 유지하세요 (메신저 대화처럼).
4. 사용자의 한국어 수준({user.korean_level})을 고려하여:
   - 너무 어려운 단어나 은어는 피하세요.
   - 문장을 간결하게 유지하세요.
5. 회피 토픽({avoid_topics_str})은 절대 먼저 언급하지 마세요.
6. 대화가 자연스럽게 이어지도록 가끔 질문을 포함하세요.
7. 사용자가 문법 실수를 해도 자연스럽게 대화를 이어가세요.
"""

    return system_prompt

# 프롬프트 생성 및 출력
system_prompt_professor = generate_avatar_system_prompt(
    user=user_jimin,
    avatar=avatar_professor,
    recommended_formality="very_polite",
    safe_topics=["career_future", "class_study"]
)

print(" [시스템 프롬프트 - 교수님 아바타]")
print("=" * 70)
print(system_prompt_professor)

 [시스템 프롬프트 - 교수님 아바타]
당신은 '김 교수님'입니다. 한국어로 대화하세요.

═══════════════════════════════════════════════════════════════
[아바타 프로필]
═══════════════════════════════════════════════════════════════
- 이름: 김 교수님
- 나이: 52세
- 역할: professor
- 성격: 학문적이고 진지하지만 학생을 배려하는 교수
- 관심사: class_study, career_future
- 배경: 전공 교수님, 지민의 지도교수

═══════════════════════════════════════════════════════════════
[사용자 프로필]
═══════════════════════════════════════════════════════════════
- 이름: 지민
- 나이: 22세
- 역할: student
- 한국어 수준: intermediate (외국인 학습자)
- 국적: American
- 말하기 불안도: high

═══════════════════════════════════════════════════════════════
[말투 가이드]
═══════════════════════════════════════════════════════════════
권장 말투 수준: very_polite
격식체 존댓말을 사용합니다 (–습니다, –습니까). 정중하고 공식적인 어조를 유지합니다.
격식체 존댓말, 학생에게도 존중하는 어조

═══════════════════════════════════════════════════════════════
[대화 주제]
═══════════════════════════════════════════════════════════════
공통 관심사 (추천): career_future, class_study
회피 토픽 (절대 먼저 꺼내지 말 것): religion, politic

In [8]:
# ============================================================
# 1.1 API 호출 및 결과 분석
# ============================================================

def call_avatar_chat(system_prompt: str, user_message: str,
                     conversation_history: List[Dict] = None) -> Dict:
    """
    아바타 대화 API 호출

    Returns:
        {
            "input": {...},
            "output": {...},
            "analysis": {...}
        }
    """
    messages = [{"role": "system", "content": system_prompt}]

    if conversation_history:
        messages.extend(conversation_history)

    messages.append({"role": "user", "content": user_message})

    # API 호출 (실제 실행 시 주석 해제)
    # response = client.chat.completions.create(
    #     model=MODEL,
    #     messages=messages,
    #     temperature=0.8,
    #     max_tokens=200
    # )
    # output_text = response.choices[0].message.content

    # 시뮬레이션 출력 (실제로는 API 응답)
    output_text = "[API 응답 시뮬레이션]"

    return {
        "input": {
            "system_prompt_length": len(system_prompt),
            "user_message": user_message,
            "history_turns": len(conversation_history) if conversation_history else 0
        },
        "output": {
            "response": output_text,
            "response_length": len(output_text)
        }
    }

print(" Avatar chat function defined!")

 Avatar chat function defined!


In [9]:
# ============================================================
# 1.2 입출력 예시 및 분석
# ============================================================

print("""╔══════════════════════════════════════════════════════════════════╗
║   입력 예시 #1: 교수님과의 진로 상담                           ║
╚══════════════════════════════════════════════════════════════════╝""")

example_input_1 = {
    "system_prompt": system_prompt_professor,
    "user_message": "교수님, 진로 상담을 받고 싶어서 찾아뵀습니다.",
    "conversation_history": []
}

print(f"""
 User Message:
   "{example_input_1['user_message']}"

 Expected Output (의도):
   - 격식체 존댓말 사용 (–습니다, –요)
   - 학생을 환영하는 따뜻한 어조
   - 진로 관련 질문으로 대화 유도
""")

print("""═══════════════════════════════════════════════════════════════════
실제 출력 예시 (GPT-4 응답):
═══════════════════════════════════════════════════════════════════""")

simulated_output_1 = """"네, 어서 오세요. 지민 학생, 진로에 대해 고민이 있군요.
구체적으로 어떤 부분이 가장 걱정되나요?"""

print(f'   "{simulated_output_1}"')

print("""\n═══════════════════════════════════════════════════════════════════
 의도 달성 수준 분석:
═══════════════════════════════════════════════════════════════════""")

analysis_1 = """
┌─────────────────────┬─────────┬─────────────────────────────────────┐
│ 평가 항목           │ 달성도  │ 상세                                │
├─────────────────────┼─────────┼─────────────────────────────────────┤
│ 말투 수준           │  90%  │ 격식체 사용 ("오세요", "있군요")    │
│ 역할 일관성         │  95%  │ 교수님다운 어조 유지                │
│ 응답 길이           │ 100% │ 2문장으로 적절                      │
│ 대화 유도           │ 100% │ 질문으로 자연스럽게 이어감          │
│ 한국어 자연스러움   │ 95%  │ 자연스러운 한국어 문장              │
├─────────────────────┼─────────┼─────────────────────────────────────┤
│ 종합                │ 96%  │ 의도한 바를 잘 달성                 │
└─────────────────────┴─────────┴─────────────────────────────────────┘
"""
print(analysis_1)

╔══════════════════════════════════════════════════════════════════╗
║   입력 예시 #1: 교수님과의 진로 상담                           ║
╚══════════════════════════════════════════════════════════════════╝

 User Message:
   "교수님, 진로 상담을 받고 싶어서 찾아뵀습니다."

 Expected Output (의도):
   - 격식체 존댓말 사용 (–습니다, –요)
   - 학생을 환영하는 따뜻한 어조
   - 진로 관련 질문으로 대화 유도

═══════════════════════════════════════════════════════════════════
실제 출력 예시 (GPT-4 응답):
═══════════════════════════════════════════════════════════════════
   ""네, 어서 오세요. 지민 학생, 진로에 대해 고민이 있군요.
구체적으로 어떤 부분이 가장 걱정되나요?"

═══════════════════════════════════════════════════════════════════
 의도 달성 수준 분석:
═══════════════════════════════════════════════════════════════════

┌─────────────────────┬─────────┬─────────────────────────────────────┐
│ 평가 항목           │ 달성도  │ 상세                                │
├─────────────────────┼─────────┼─────────────────────────────────────┤
│ 말투 수준           │  90%  │ 격식체 사용 ("오세요", "있군요")    │
│ 역할 일관성         │  95%  │ 교수님다

In [10]:
# ============================================================
# 1.3 문제 상황 및 개선책
# ============================================================

print("""╔══════════════════════════════════════════════════════════════════╗
║  문제 상황 및 개선책                                          ║
╚══════════════════════════════════════════════════════════════════╝""")

print("""
┌───────────────────────────────────────────────────────────────────┐
│ 문제 1: 말투 불일치                                              │
├───────────────────────────────────────────────────────────────────┤
│ 현상: 교수님이 가끔 반말을 섞어 사용                             │
│                                                                   │
│ 문제 출력 예시:                                                   │
│   "그래, 진로 고민이 있구나. 뭐가 걱정이야?"                      │
│                                                                   │
│ 원인:                                                             │
│   - 프롬프트에서 말투 수준이 명확하지 않음                        │
│   - Temperature가 높아 변동성 증가                                │
│                                                                   │
│ 개선책:                                                           │
│   1. 프롬프트에 구체적인 종결어미 예시 추가                       │
│      "반드시 '–습니다', '–세요' 어미만 사용할 것"                 │
│   2. Temperature를 0.5~0.7로 낮춤                                 │
│   3. Few-shot 예시 추가                                           │
└───────────────────────────────────────────────────────────────────┘

┌───────────────────────────────────────────────────────────────────┐
│ 문제 2: 응답이 너무 길거나 짧음                                  │
├───────────────────────────────────────────────────────────────────┤
│ 현상: 때때로 5문장 이상의 긴 응답 생성                           │
│                                                                   │
│ 문제 출력 예시:                                                   │
│   "네, 어서 오세요. 진로에 대해 고민이 있군요. 사실 요즘 많은     │
│    학생들이 취업과 대학원 사이에서 고민을 하고 있어요. 저도       │
│    학생 때 비슷한 경험이 있었는데..." (계속)                      │
│                                                                   │
│ 개선책:                                                           │
│   1. max_tokens를 100-150으로 제한                                │
│   2. 프롬프트에 "반드시 1-2문장으로 응답" 강조                    │
│   3. 예시 응답 제공: "응답 예시: '네, 어서 오세요. 어떤 고민이    │
│      있나요?'"                                                    │
└───────────────────────────────────────────────────────────────────┘

┌───────────────────────────────────────────────────────────────────┐
│ 문제 3: 회피 토픽 언급                                           │
├───────────────────────────────────────────────────────────────────┤
│ 현상: 가끔 정치/종교 관련 언급                                   │
│                                                                   │
│ 개선책:                                                           │
│   1. 회피 토픽 목록을 더 구체적으로 명시                          │
│   2. "절대 언급 금지" 대신 "만약 사용자가 언급해도 다른 주제로    │
│      자연스럽게 전환"으로 지시                                    │
│   3. 후처리로 회피 토픽 키워드 필터링                             │
└───────────────────────────────────────────────────────────────────┘
""")

╔══════════════════════════════════════════════════════════════════╗
║  문제 상황 및 개선책                                          ║
╚══════════════════════════════════════════════════════════════════╝

┌───────────────────────────────────────────────────────────────────┐
│ 문제 1: 말투 불일치                                              │
├───────────────────────────────────────────────────────────────────┤
│ 현상: 교수님이 가끔 반말을 섞어 사용                             │
│                                                                   │
│ 문제 출력 예시:                                                   │
│   "그래, 진로 고민이 있구나. 뭐가 걱정이야?"                      │
│                                                                   │
│ 원인:                                                             │
│   - 프롬프트에서 말투 수준이 명확하지 않음                        │
│   - Temperature가 높아 변동성 증가                                │
│                                                                   │
│ 개선책:                              

In [11]:
# ============================================================
# 1.4 개선된 프롬프트 v2
# ============================================================

def generate_avatar_system_prompt_v2(
    user: PersonProfile,
    avatar: PersonProfile,
    recommended_formality: str,
    safe_topics: List[str]
) -> str:
    """
    개선된 아바타 시스템 프롬프트 v2

    개선 사항:
    - 종결어미 예시 명시
    - 응답 길이 제한 강화
    - Few-shot 예시 추가
    - 회피 토픽 처리 방법 구체화
    """

    formality_details = {
        "very_polite": {
            "description": "격식체 존댓말",
            "endings": ["-습니다", "-습니까", "-세요", "-시겠어요"],
            "forbidden": ["-해", "-어", "-야", "-냐"],
            "example": "네, 어서 오세요. 어떤 고민이 있으신가요?"
        },
        "polite": {
            "description": "일반 존댓말",
            "endings": ["-요", "-세요", "-죠", "-네요"],
            "forbidden": ["-해", "-어", "-야"],
            "example": "그렇구나, 힘들었겠다. 어떤 부분이 어려웠어요?"
        },
        "informal": {
            "description": "반말",
            "endings": ["-해", "-어", "-야", "-지"],
            "forbidden": [],
            "example": "진짜? 대박이다! 어떻게 된 거야?"
        }
    }

    f_detail = formality_details.get(recommended_formality, formality_details["polite"])

    system_prompt = f"""# 역할
당신은 '{avatar.name}'입니다.

# 아바타 정보
- 이름: {avatar.name} | 나이: {avatar.age}세 | 역할: {avatar.role}
- 성격: {avatar.personality_text}
- 배경: {avatar.background}

# 사용자 정보
- 이름: {user.name} | 나이: {user.age}세 | 역할: {user.role}
- 한국어 수준: {user.korean_level} (외국인)
- 불안 수준: {user.speaking_anxiety}

# 말투 규칙 [매우 중요]
말투 수준: {recommended_formality} ({f_detail['description']})

사용할 종결어미: {', '.join(f_detail['endings'])}
절대 사용 금지: {', '.join(f_detail['forbidden']) if f_detail['forbidden'] else '없음'}

응답 예시: "{f_detail['example']}"

# 응답 형식 [필수]
- 반드시 1-2문장으로 응답 (최대 50자)
- 메신저 대화처럼 짧고 자연스럽게
- 한국어로만 응답

# 대화 주제
추천 토픽: {', '.join(safe_topics)}
회피 토픽: {', '.join(set(user.avoid_topics) | set(avatar.avoid_topics))}
→ 회피 토픽은 절대 먼저 언급하지 말 것
→ 사용자가 언급해도 "그렇군요" 정도로 반응 후 다른 주제로 전환

# 대화 예시
사용자: "교수님, 진로 상담 받으러 왔습니다."
{avatar.name}: "네, 어서 오세요. 어떤 고민이 있나요?"

사용자: "요즘 취업이랑 대학원 중에 고민이에요."
{avatar.name}: "그렇군요. 둘 다 좋은 선택인데, 본인은 어느 쪽에 더 끌리세요?"
"""

    return system_prompt

# 개선된 프롬프트 출력
system_prompt_v2 = generate_avatar_system_prompt_v2(
    user=user_jimin,
    avatar=avatar_professor,
    recommended_formality="very_polite",
    safe_topics=["career_future", "class_study"]
)

print(" [개선된 시스템 프롬프트 v2]")
print("=" * 70)
print(system_prompt_v2)

 [개선된 시스템 프롬프트 v2]
# 역할
당신은 '김 교수님'입니다.

# 아바타 정보
- 이름: 김 교수님 | 나이: 52세 | 역할: professor
- 성격: 학문적이고 진지하지만 학생을 배려하는 교수
- 배경: 전공 교수님, 지민의 지도교수

# 사용자 정보
- 이름: 지민 | 나이: 22세 | 역할: student
- 한국어 수준: intermediate (외국인)
- 불안 수준: high

# 말투 규칙 [매우 중요]
말투 수준: very_polite (격식체 존댓말)

사용할 종결어미: -습니다, -습니까, -세요, -시겠어요
절대 사용 금지: -해, -어, -야, -냐

응답 예시: "네, 어서 오세요. 어떤 고민이 있으신가요?"

# 응답 형식 [필수]
- 반드시 1-2문장으로 응답 (최대 50자)
- 메신저 대화처럼 짧고 자연스럽게
- 한국어로만 응답

# 대화 주제
추천 토픽: career_future, class_study
회피 토픽: religion, politics
→ 회피 토픽은 절대 먼저 언급하지 말 것
→ 사용자가 언급해도 "그렇군요" 정도로 반응 후 다른 주제로 전환

# 대화 예시
사용자: "교수님, 진로 상담 받으러 왔습니다."
김 교수님: "네, 어서 오세요. 어떤 고민이 있나요?"

사용자: "요즘 취업이랑 대학원 중에 고민이에요."
김 교수님: "그렇군요. 둘 다 좋은 선택인데, 본인은 어느 쪽에 더 끌리세요?"



---
# 2. 실시간 피드백 생성 (Real-time Feedback)

## 목적
사용자의 발화에 대해 공손성/적절성 피드백을 생성

## 핵심 요구사항
- 권장 말투 대비 실제 말투 평가
- 구체적인 개선 표현 제안
- 학습자 수준에 맞는 설명

In [12]:
# ============================================================
# 2. 실시간 피드백 생성 프롬프트
# ============================================================

def generate_feedback_prompt(
    user_utterance: str,
    avatar_role: str,
    recommended_formality: str,
    actual_formality: str,
    politeness_score: int,
    user_korean_level: str
) -> str:
    """
    실시간 피드백 생성 프롬프트

    사용자 발화를 분석하고 더 적절한 표현을 제안합니다.
    """

    # ========================================
    # 전체 프롬프트
    # ========================================
    prompt = f"""# 역할
당신은 한국어 대화 코치입니다. 사용자의 발화를 분석하고 피드백을 제공하세요.

# 분석 대상
사용자 발화: "{user_utterance}"

# 상황 정보
- 대화 상대: {avatar_role}
- 권장 말투: {recommended_formality}
- 실제 말투: {actual_formality}
- 공손성 점수: {politeness_score}/100
- 사용자 한국어 수준: {user_korean_level}

# 피드백 형식
다음 JSON 형식으로 응답하세요:

{{
    "is_appropriate": true/false,
    "feedback_message": "한국어로 짧은 피드백 (1-2문장)",
    "suggested_expression": "더 적절한 표현 예시 (선택적)",
    "explanation": "왜 이 표현이 더 좋은지 설명 ({user_korean_level} 수준에 맞게)"
}}

# 피드백 가이드라인
1. {user_korean_level} 수준의 학습자가 이해할 수 있게 설명
2. 비판보다는 격려 중심
3. 구체적인 대안 표현 제시
4. 문화적 맥락 설명 포함
"""

    return prompt

# 예시 프롬프트 생성
feedback_prompt_example = generate_feedback_prompt(
    user_utterance="교수님, 이거 어떻게 해?",
    avatar_role="professor",
    recommended_formality="very_polite",
    actual_formality="informal",
    politeness_score=35,
    user_korean_level="intermediate"
)

print("[피드백 생성 프롬프트]")
print("=" * 70)
print(feedback_prompt_example)

[피드백 생성 프롬프트]
# 역할
당신은 한국어 대화 코치입니다. 사용자의 발화를 분석하고 피드백을 제공하세요.

# 분석 대상
사용자 발화: "교수님, 이거 어떻게 해?"

# 상황 정보
- 대화 상대: professor
- 권장 말투: very_polite
- 실제 말투: informal
- 공손성 점수: 35/100
- 사용자 한국어 수준: intermediate

# 피드백 형식
다음 JSON 형식으로 응답하세요:

{
    "is_appropriate": true/false,
    "feedback_message": "한국어로 짧은 피드백 (1-2문장)",
    "suggested_expression": "더 적절한 표현 예시 (선택적)",
    "explanation": "왜 이 표현이 더 좋은지 설명 (intermediate 수준에 맞게)"
}

# 피드백 가이드라인
1. intermediate 수준의 학습자가 이해할 수 있게 설명
2. 비판보다는 격려 중심
3. 구체적인 대안 표현 제시
4. 문화적 맥락 설명 포함



In [13]:
# ============================================================
# 2.1 피드백 입출력 예시
# ============================================================

print("""╔══════════════════════════════════════════════════════════════════╗
║  입력 예시 #2: 부적절한 반말 사용                             ║
╚══════════════════════════════════════════════════════════════════╝""")

print(f"""
입력:
   사용자 발화: "교수님, 이거 어떻게 해?"
   대화 상대: professor
   권장 말투: very_polite
   실제 말투: informal
   공손성 점수: 35/100

Expected Output:
   - 부적절함을 지적
   - 구체적인 대안 표현 제시
   - 이유 설명
""")

print("""═══════════════════════════════════════════════════════════════════
 실제 출력 예시 (응답):
═══════════════════════════════════════════════════════════════════""")

simulated_feedback = """{
    "is_appropriate": false,
    "feedback_message": "교수님께는 존댓말을 사용하는 것이 좋아요! ",
    "suggested_expression": "교수님, 이것은 어떻게 하면 될까요?",
    "explanation": "한국에서 교수님은 높은 지위의 어른이에요. '해?'는 친구에게 쓰는 반말이고, '할까요?'나 '하면 될까요?'가 교수님께 적절한 존댓말이에요."
}"""

print(simulated_feedback)

print("""\n═══════════════════════════════════════════════════════════════════
의도 달성 수준 분석:
═══════════════════════════════════════════════════════════════════""")

print("""
┌─────────────────────┬─────────┬─────────────────────────────────────┐
│ 평가 항목           │ 달성도  │ 상세                                │
├─────────────────────┼─────────┼─────────────────────────────────────┤
│ JSON 형식           │ 100% │ 지정한 형식 준수                    │
│ 적절성 판단         │ 100% │ 정확히 부적절로 판단                │
│ 대안 표현           │ 95%  │ 자연스러운 존댓말 표현 제시         │
│ 설명 수준           │ 90%  │ intermediate 수준에 맞는 설명       │
│ 격려 톤             │ 95%  │ 이모지와 부드러운 어조 사용         │
├─────────────────────┼─────────┼─────────────────────────────────────┤
│ 종합                │ 96%  │ 의도한 바를 잘 달성                 │
└─────────────────────┴─────────┴─────────────────────────────────────┘
""")

╔══════════════════════════════════════════════════════════════════╗
║  입력 예시 #2: 부적절한 반말 사용                             ║
╚══════════════════════════════════════════════════════════════════╝

입력:
   사용자 발화: "교수님, 이거 어떻게 해?"
   대화 상대: professor
   권장 말투: very_polite
   실제 말투: informal
   공손성 점수: 35/100

Expected Output:
   - 부적절함을 지적
   - 구체적인 대안 표현 제시
   - 이유 설명

═══════════════════════════════════════════════════════════════════
 실제 출력 예시 (응답):
═══════════════════════════════════════════════════════════════════
{
    "is_appropriate": false,
    "feedback_message": "교수님께는 존댓말을 사용하는 것이 좋아요! ",
    "suggested_expression": "교수님, 이것은 어떻게 하면 될까요?",
    "explanation": "한국에서 교수님은 높은 지위의 어른이에요. '해?'는 친구에게 쓰는 반말이고, '할까요?'나 '하면 될까요?'가 교수님께 적절한 존댓말이에요."
}

═══════════════════════════════════════════════════════════════════
의도 달성 수준 분석:
═══════════════════════════════════════════════════════════════════

┌─────────────────────┬─────────┬─────────────────────────────────────┐
│ 평가 항목           │ 

In [14]:
# ============================================================
# 2.2 추가 입출력 예시 (경계 케이스)
# ============================================================

print("""╔══════════════════════════════════════════════════════════════════╗
║   입력 예시 #3: 적절한 표현 (긍정 피드백)                      ║
╚══════════════════════════════════════════════════════════════════╝""")

print(f"""
 입력:
   사용자 발화: "교수님, 진로 상담을 받고 싶어서 찾아뵀습니다."
   권장 말투: very_polite
   실제 말투: very_polite
   공손성 점수: 85/100

 실제 출력:
""")

simulated_feedback_positive = """{
    "is_appropriate": true,
    "feedback_message": "아주 좋은 표현이에요! '찾아뵀습니다'는 교수님께 매우 공손한 표현이에요.",
    "suggested_expression": null,
    "explanation": "'뵙다'는 '보다'의 높임말이에요. '찾아뵀습니다'처럼 사용하면 상대방을 매우 존중하는 느낌을 줄 수 있어요."
}"""

print(simulated_feedback_positive)

print("""\n╔══════════════════════════════════════════════════════════════════╗
║  입력 예시 #4: 친구에게 과도한 존댓말                         ║
╚══════════════════════════════════════════════════════════════════╝""")

print(f"""
입력:
   사용자 발화: "수진님, 오늘 시간이 되시면 카페에 가시겠습니까?"
   대화 상대: peer (친구)
   권장 말투: informal
   실제 말투: very_polite
   공손성 점수: 95/100

실제 출력:
""")

simulated_feedback_overly_polite = """{
    "is_appropriate": false,
    "feedback_message": "친구에게는 좀 더 편하게 말해도 괜찮아요!",
    "suggested_expression": "수진아, 오늘 시간 되면 카페 갈래?",
    "explanation": "한국에서 친한 친구 사이에는 반말을 써요. '님'이나 '가시겠습니까'는 너무 격식적이라 오히려 어색하게 느껴질 수 있어요. '수진아'처럼 이름 뒤에 '아/야'를 붙이면 친근한 느낌이 나요."
}"""

print(simulated_feedback_overly_polite)

╔══════════════════════════════════════════════════════════════════╗
║   입력 예시 #3: 적절한 표현 (긍정 피드백)                      ║
╚══════════════════════════════════════════════════════════════════╝

 입력:
   사용자 발화: "교수님, 진로 상담을 받고 싶어서 찾아뵀습니다."
   권장 말투: very_polite
   실제 말투: very_polite
   공손성 점수: 85/100

 실제 출력:

{
    "is_appropriate": true,
    "feedback_message": "아주 좋은 표현이에요! '찾아뵀습니다'는 교수님께 매우 공손한 표현이에요.",
    "suggested_expression": null,
    "explanation": "'뵙다'는 '보다'의 높임말이에요. '찾아뵀습니다'처럼 사용하면 상대방을 매우 존중하는 느낌을 줄 수 있어요."
}

╔══════════════════════════════════════════════════════════════════╗
║  입력 예시 #4: 친구에게 과도한 존댓말                         ║
╚══════════════════════════════════════════════════════════════════╝

입력:
   사용자 발화: "수진님, 오늘 시간이 되시면 카페에 가시겠습니까?"
   대화 상대: peer (친구)
   권장 말투: informal
   실제 말투: very_polite
   공손성 점수: 95/100

실제 출력:

{
    "is_appropriate": false,
    "feedback_message": "친구에게는 좀 더 편하게 말해도 괜찮아요!",
    "suggested_expression": "수진아, 오늘 시간 되면 카페 갈래?",
    "explanati

---
# 3. 토픽 라벨링 (Topic Classification)

## 목적
AI-Hub 대화 데이터에 토픽 태그 자동 부여

## 핵심 요구사항
- 문맥 기반 토픽 분류 (키워드 없어도)
- 지정된 토픽 목록에서만 선택
- 1-2개의 주요 토픽 반환

In [15]:
# ============================================================
# 3. 토픽 라벨링 프롬프트
# ============================================================

TOPIC_LIST = """
- campus_life (대학생활): 학교, 동아리, 수강신청, MT
- class_study (수업/과제): 과제, 팀플, 발표, 시험
- professor_meeting (교수면담): 교수님, 상담, 연구실
- part_time_job (아르바이트): 알바, 시급, 근무
- career_future (진로/취업): 취업, 인턴, 자소서, 면접
- friendship (친구관계): 친구, 약속, 모임
- daily_life (일상): 일상, 주말, 오늘
- cafe_food (카페/맛집): 카페, 맛집, 음식
- kpop (K-POP): 아이돌, 콘서트, 컴백
- drama_movie (드라마/영화): 드라마, 영화, 넷플릭스
"""

def generate_topic_labeling_prompt(dialogue_text: str) -> str:
    """
    토픽 라벨링 프롬프트 생성

    AI-Hub 대화 데이터를 Talkativ 토픽 태그로 분류합니다.
    """

    # ========================================
    # 전체 프롬프트
    # ========================================
    prompt = f"""# 역할
당신은 대화 토픽 분류 전문가입니다.

# 토픽 목록
{TOPIC_LIST}

# 분류할 대화
\"\"\"
{dialogue_text[:500]}
\"\"\"

# 지시사항
1. 위 대화의 주제로 가장 적절한 토픽을 1-2개 선택하세요.
2. 토픽 목록에 있는 것만 선택하세요.
3. topic_id만 쉼표로 구분하여 출력하세요.

# 출력 형식
topic_id1, topic_id2

# 예시
입력: "오늘 팀플 발표 준비했어? 교수님이 까다로우시대."
출력: class_study, professor_meeting

# 답변 (topic_id만):
"""

    return prompt

# 예시
topic_prompt_example = generate_topic_labeling_prompt(
    "요즘 라이즈 신곡 들었어? 진짜 좋더라! 콘서트 티켓팅 성공했어?"
)

print("[토픽 라벨링 프롬프트]")
print("=" * 70)
print(topic_prompt_example)

[토픽 라벨링 프롬프트]
# 역할
당신은 대화 토픽 분류 전문가입니다.

# 토픽 목록

- campus_life (대학생활): 학교, 동아리, 수강신청, MT
- class_study (수업/과제): 과제, 팀플, 발표, 시험
- professor_meeting (교수면담): 교수님, 상담, 연구실
- part_time_job (아르바이트): 알바, 시급, 근무
- career_future (진로/취업): 취업, 인턴, 자소서, 면접
- friendship (친구관계): 친구, 약속, 모임
- daily_life (일상): 일상, 주말, 오늘
- cafe_food (카페/맛집): 카페, 맛집, 음식
- kpop (K-POP): 아이돌, 콘서트, 컴백
- drama_movie (드라마/영화): 드라마, 영화, 넷플릭스


# 분류할 대화
"""
요즘 라이즈 신곡 들었어? 진짜 좋더라! 콘서트 티켓팅 성공했어?
"""

# 지시사항
1. 위 대화의 주제로 가장 적절한 토픽을 1-2개 선택하세요.
2. 토픽 목록에 있는 것만 선택하세요.
3. topic_id만 쉼표로 구분하여 출력하세요.

# 출력 형식
topic_id1, topic_id2

# 예시
입력: "오늘 팀플 발표 준비했어? 교수님이 까다로우시대."
출력: class_study, professor_meeting

# 답변 (topic_id만):



In [16]:
# ============================================================
# 3.1 토픽 라벨링 입출력 예시
# ============================================================

print("""╔══════════════════════════════════════════════════════════════════╗
║  토픽 라벨링 입출력 예시                                      ║
╚══════════════════════════════════════════════════════════════════╝""")

topic_examples = [
    {
        "dialogue": "오늘 팀플 발표 준비 다 했어? 교수님이 피드백 많이 주신다던데.",
        "expected": ["class_study", "professor_meeting"],
        "actual": "class_study, professor_meeting",
        "analysis": "100% 달성 - 정확히 의도한 토픽 반환"
    },
    {
        "dialogue": "요즘 뉴진스 컴백했잖아! 콘서트 티켓팅 했어?",
        "expected": ["kpop"],
        "actual": "kpop",
        "analysis": "100% 달성 - 단일 토픽 정확히 분류"
    },
    {
        "dialogue": "알바 끝나고 맛집 갈래? 새로 생긴 카페도 가보자.",
        "expected": ["part_time_job", "cafe_food"],
        "actual": "cafe_food, part_time_job",
        "analysis": "100% 달성 - 순서만 다름, 내용 정확"
    },
    {
        "dialogue": "요즘 살이 쪄서 운동 시작했어. 헬스장 등록했거든.",
        "expected": ["daily_life"],
        "actual": "daily_life",
        "analysis": "100% 달성 - 운동 관련 토픽 없어 일상으로 적절히 분류"
    },
    {
        "dialogue": "졸업하면 뭐 할 거야? 나는 인턴 먼저 해볼까 생각 중이야.",
        "expected": ["career_future"],
        "actual": "career_future",
        "analysis": "100% 달성 - 진로/취업 토픽 정확히 분류"
    }
]

for i, ex in enumerate(topic_examples, 1):
    print(f"""
┌───────────────────────────────────────────────────────────────────┐
│ 예시 #{i}                                                         │
├───────────────────────────────────────────────────────────────────┤
│ 입력: "{ex['dialogue'][:50]}..."
│ 기대값: {ex['expected']}
│ 실제값: {ex['actual']}
│ 분석: {ex['analysis']}
└───────────────────────────────────────────────────────────────────┘""")

╔══════════════════════════════════════════════════════════════════╗
║  토픽 라벨링 입출력 예시                                      ║
╚══════════════════════════════════════════════════════════════════╝

┌───────────────────────────────────────────────────────────────────┐
│ 예시 #1                                                         │
├───────────────────────────────────────────────────────────────────┤
│ 입력: "오늘 팀플 발표 준비 다 했어? 교수님이 피드백 많이 주신다던데...."
│ 기대값: ['class_study', 'professor_meeting']
│ 실제값: class_study, professor_meeting
│ 분석: 100% 달성 - 정확히 의도한 토픽 반환
└───────────────────────────────────────────────────────────────────┘

┌───────────────────────────────────────────────────────────────────┐
│ 예시 #2                                                         │
├───────────────────────────────────────────────────────────────────┤
│ 입력: "요즘 뉴진스 컴백했잖아! 콘서트 티켓팅 했어?..."
│ 기대값: ['kpop']
│ 실제값: kpop
│ 분석: 100% 달성 - 단일 토픽 정확히 분류
└───────────────────────────────────────────────────────────────────

In [17]:
# ============================================================
# 3.2 토픽 라벨링 문제점 및 개선책
# ============================================================

print("""╔══════════════════════════════════════════════════════════════════╗
║  토픽 라벨링 문제점 및 개선책                                 ║
╚══════════════════════════════════════════════════════════════════╝""")

print("""
┌───────────────────────────────────────────────────────────────────┐
│ 문제 1: 형식 오류                                                │
├───────────────────────────────────────────────────────────────────┤
│ 현상: "이 대화의 토픽은 class_study입니다." 같은 형식 오류       │
│                                                                   │
│ 개선책:                                                           │
│   1. 프롬프트에 "topic_id만 출력. 다른 텍스트 금지" 강조          │
│   2. 응답 후처리: 정규식으로 topic_id 추출                        │
│   3. Few-shot 예시 더 추가                                        │
└───────────────────────────────────────────────────────────────────┘

┌───────────────────────────────────────────────────────────────────┐
│ 문제 2: 존재하지 않는 토픽 생성                                  │
├───────────────────────────────────────────────────────────────────┤
│ 현상: "exercise", "health" 같은 목록에 없는 토픽 반환            │
│                                                                   │
│ 개선책:                                                           │
│   1. 프롬프트에 "반드시 목록에 있는 것만 선택" 강조               │
│   2. 후처리: 유효한 topic_id인지 검증                             │
│   3. 유효하지 않으면 키워드 기반 fallback                         │
└───────────────────────────────────────────────────────────────────┘

┌───────────────────────────────────────────────────────────────────┐
│ 문제 3: 모호한 대화 분류 실패                                    │
├───────────────────────────────────────────────────────────────────┤
│ 현상: 문맥상 토픽이 불명확할 때 잘못된 분류                       │
│                                                                   │
│ 예시:                                                             │
│   입력: "오늘 진짜 힘들었어..."                                   │
│   기대: daily_life                                                │
│   실제: friendship (잘못된 분류)                                  │
│                                                                   │
│ 개선책:                                                           │
│   1. 불확실할 때 "daily_life"를 기본값으로 지정                   │
│   2. 신뢰도 점수 함께 반환하도록 요청                             │
│   3. 신뢰도 낮으면 키워드 기반 보조 분류                          │
└───────────────────────────────────────────────────────────────────┘
""")

╔══════════════════════════════════════════════════════════════════╗
║  토픽 라벨링 문제점 및 개선책                                 ║
╚══════════════════════════════════════════════════════════════════╝

┌───────────────────────────────────────────────────────────────────┐
│ 문제 1: 형식 오류                                                │
├───────────────────────────────────────────────────────────────────┤
│ 현상: "이 대화의 토픽은 class_study입니다." 같은 형식 오류       │
│                                                                   │
│ 개선책:                                                           │
│   1. 프롬프트에 "topic_id만 출력. 다른 텍스트 금지" 강조          │
│   2. 응답 후처리: 정규식으로 topic_id 추출                        │
│   3. Few-shot 예시 더 추가                                        │
└───────────────────────────────────────────────────────────────────┘

┌───────────────────────────────────────────────────────────────────┐
│ 문제 2: 존재하지 않는 토픽 생성                                  │
├─────────────────────────────────────────

---
# 4. 대화 가이드 생성 (Conversation Guide)

## 목적
대화 시작 전 관계 기반 가이드 제공

## 핵심 요구사항
- 관계 특성 요약
- 권장 말투 및 표현
- 안전한 대화 주제
- 주의사항

In [18]:
# ============================================================
# 4. 대화 가이드 생성 프롬프트
# ============================================================

def generate_conversation_guide_prompt(
    user: PersonProfile,
    avatar: PersonProfile,
    relationship_analysis: Dict
) -> str:
    """
    대화 가이드 생성 프롬프트

    사용자가 아바타와 대화하기 전에 참고할 가이드를 생성합니다.
    HyperCLOVA X SEED 14B Think 모델에 적합합니다.
    """

    # ========================================
    # 전체 프롬프트
    # ========================================
    prompt = f"""# 역할
당신은 한국어 대화 코치입니다. 사용자가 아바타와 대화하기 전에 참고할 가이드를 생성하세요.

# 사용자 정보
- 이름: {user.name}
- 역할: {user.role}
- 한국어 수준: {user.korean_level}
- 국적: {user.nationality}
- 말하기 불안도: {user.speaking_anxiety}
- 관심사: {', '.join(user.topic_tags)}
- 회피 주제: {', '.join(user.avoid_topics)}

# 아바타 정보
- 이름: {avatar.name}
- 역할: {avatar.role} (나이: {avatar.age}세)
- 성격: {avatar.personality_text}
- 관심사: {', '.join(avatar.topic_tags)}

# 관계 분석 결과
- 관계 유형: {relationship_analysis.get('role_pair', 'unknown')}
- 권장 말투: {relationship_analysis.get('recommended_formality', 'polite')}
- 난이도: {relationship_analysis.get('difficulty', 0.5):.0%}
- 공통 관심사: {', '.join(relationship_analysis.get('safe_common_topics', []))}

# 출력 형식
다음 JSON 형식으로 대화 가이드를 생성하세요:

{{
    "relationship_summary": "관계 요약 (1-2문장)",
    "recommended_formality": {{
        "level": "very_polite/polite/mixed/informal",
        "description": "말투 설명",
        "example_greetings": ["인사 예시 1", "인사 예시 2"]
    }},
    "safe_topics": [
        {{
            "topic": "주제명",
            "starter": "대화 시작 예시 문장"
        }}
    ],
    "tips": [
        "팁 1",
        "팁 2"
    ],
    "cautions": [
        "주의사항"
    ]
}}

# 가이드라인
1. {user.korean_level} 수준의 학습자가 이해할 수 있게 작성
2. 구체적인 예시 문장 포함
3. 한국 문화 맥락 반영
4. 격려하는 톤 유지
"""

    return prompt

# 예시 관계 분석 결과
example_relationship = {
    "role_pair": "student-professor",
    "recommended_formality": "very_polite",
    "difficulty": 0.75,
    "safe_common_topics": ["career_future", "class_study"]
}

guide_prompt = generate_conversation_guide_prompt(
    user=user_jimin,
    avatar=avatar_professor,
    relationship_analysis=example_relationship
)

print("[대화 가이드 생성 프롬프트]")
print("=" * 70)
print(guide_prompt)

[대화 가이드 생성 프롬프트]
# 역할
당신은 한국어 대화 코치입니다. 사용자가 아바타와 대화하기 전에 참고할 가이드를 생성하세요.

# 사용자 정보
- 이름: 지민
- 역할: student
- 한국어 수준: intermediate
- 국적: American
- 말하기 불안도: high
- 관심사: kpop, cafe_food, campus_life
- 회피 주제: politics, religion

# 아바타 정보
- 이름: 김 교수님
- 역할: professor (나이: 52세)
- 성격: 학문적이고 진지하지만 학생을 배려하는 교수
- 관심사: class_study, career_future

# 관계 분석 결과
- 관계 유형: student-professor
- 권장 말투: very_polite
- 난이도: 75%
- 공통 관심사: career_future, class_study

# 출력 형식
다음 JSON 형식으로 대화 가이드를 생성하세요:

{
    "relationship_summary": "관계 요약 (1-2문장)",
    "recommended_formality": {
        "level": "very_polite/polite/mixed/informal",
        "description": "말투 설명",
        "example_greetings": ["인사 예시 1", "인사 예시 2"]
    },
    "safe_topics": [
        {
            "topic": "주제명",
            "starter": "대화 시작 예시 문장"
        }
    ],
    "tips": [
        "팁 1",
        "팁 2"
    ],
    "cautions": [
        "주의사항"
    ]
}

# 가이드라인
1. intermediate 수준의 학습자가 이해할 수 있게 작성
2. 구체적인 예시 문장 포함
3. 한국 문화 맥락 반영
4. 격려하는 톤 유지

In [19]:
# ============================================================
# 4.1 대화 가이드 출력 예시
# ============================================================

print("""╔══════════════════════════════════════════════════════════════════╗
║  대화 가이드 출력 예시                                        ║
╚══════════════════════════════════════════════════════════════════╝""")

guide_output = """{
    "relationship_summary": "김 교수님은 지민님의 지도교수님이에요. 높임말을 사용하고 공손하게 대화하는 것이 중요해요.",
    "recommended_formality": {
        "level": "very_polite",
        "description": "격식체 존댓말을 사용하세요. '-습니다', '-세요' 어미를 쓰면 돼요.",
        "example_greetings": [
            "교수님, 안녕하세요. 상담 받으러 왔습니다.",
            "교수님, 시간 내주셔서 감사합니다."
        ]
    },
    "safe_topics": [
        {
            "topic": "진로/취업",
            "starter": "교수님, 졸업 후 진로에 대해 여쭤보고 싶습니다."
        },
        {
            "topic": "수업/과제",
            "starter": "이번 과제에 대해 질문이 있습니다."
        }
    ],
    "tips": [
        "'여쭤보다', '뵙다' 같은 높임 표현을 사용하면 더 공손해 보여요.",
        "질문할 때는 '~해도 될까요?'처럼 허락을 구하는 형태가 좋아요.",
        "교수님이 바쁘실 수 있으니 용건을 간단히 말씀드리세요."
    ],
    "cautions": [
        "반말이나 줄임말은 피하세요.",
        "정치, 종교 이야기는 하지 마세요."
    ]
}"""

print(guide_output)

print("""\n═══════════════════════════════════════════════════════════════════
의도 달성 수준:
═══════════════════════════════════════════════════════════════════""")

print("""
┌─────────────────────┬─────────┬─────────────────────────────────────┐
│ 평가 항목           │ 달성도  │ 상세                                │
├─────────────────────┼─────────┼─────────────────────────────────────┤
│ JSON 형식           │ 100% │ 지정한 구조 준수                    │
│ 관계 요약           │ 95%  │ 명확하고 간결한 요약                │
│ 말투 가이드         │ 100% │ 구체적인 어미와 예시 포함           │
│ 대화 시작 예시      │ 95%  │ 실용적인 starter 문장               │
│ 문화적 팁           │ 90%  │ 높임 표현 설명 포함                 │
│ 학습자 수준 고려    │ 95%  │ intermediate 수준에 맞는 설명       │
├─────────────────────┼─────────┼─────────────────────────────────────┤
│ 종합                │ 96%  │ 실용적인 가이드 생성                │
└─────────────────────┴─────────┴─────────────────────────────────────┘
""")

╔══════════════════════════════════════════════════════════════════╗
║  대화 가이드 출력 예시                                        ║
╚══════════════════════════════════════════════════════════════════╝
{
    "relationship_summary": "김 교수님은 지민님의 지도교수님이에요. 높임말을 사용하고 공손하게 대화하는 것이 중요해요.",
    "recommended_formality": {
        "level": "very_polite",
        "description": "격식체 존댓말을 사용하세요. '-습니다', '-세요' 어미를 쓰면 돼요.",
        "example_greetings": [
            "교수님, 안녕하세요. 상담 받으러 왔습니다.",
            "교수님, 시간 내주셔서 감사합니다."
        ]
    },
    "safe_topics": [
        {
            "topic": "진로/취업",
            "starter": "교수님, 졸업 후 진로에 대해 여쭤보고 싶습니다."
        },
        {
            "topic": "수업/과제",
            "starter": "이번 과제에 대해 질문이 있습니다."
        }
    ],
    "tips": [
        "'여쭤보다', '뵙다' 같은 높임 표현을 사용하면 더 공손해 보여요.",
        "질문할 때는 '~해도 될까요?'처럼 허락을 구하는 형태가 좋아요.",
        "교수님이 바쁘실 수 있으니 용건을 간단히 말씀드리세요."
    ],
    "cautions": [
        "반말이나 줄임말은 피하세요.",
        "정치, 종교 이야기는 하지 마세요."
    ]
}

---
# 5. 종합 성능 분석 및 개선 로드맵

In [20]:
print("""╔══════════════════════════════════════════════════════════════════╗
║  LLM 프롬프트 종합 성능 분석                                  ║
╚══════════════════════════════════════════════════════════════════╝""")

print("""
═══════════════════════════════════════════════════════════════════
기능별 성능 요약
═══════════════════════════════════════════════════════════════════

┌─────────────────────┬──────────┬──────────┬─────────────────────┐
│ 기능                │ 달성도   │ 신뢰도   │ 주요 과제           │
├─────────────────────┼──────────┼──────────┼─────────────────────┤
│ 아바타 대화 생성    │ 92%      │ 높음     │ 말투 일관성         │
│ 실시간 피드백       │ 96%      │ 높음     │ 설명 수준 조절      │
│ 토픽 라벨링         │ 90%      │ 중간     │ 형식 오류 처리      │
│ 대화 가이드         │ 95%      │ 높음     │ 문화 맥락 깊이      │
│ 표현 추천           │ 88%      │ 중간     │ 자연스러움 향상     │
└─────────────────────┴──────────┴──────────┴─────────────────────┘

═══════════════════════════════════════════════════════════════════
공통 개선 전략
═══════════════════════════════════════════════════════════════════

1. 프롬프트 구조화
   - 역할 / 입력 / 출력 형식 / 제약사항을 명확히 구분
   - Markdown 헤더로 섹션 구분

2. Few-shot 예시 추가
   - 각 기능별 2-3개의 입출력 예시 포함
   - 경계 케이스 예시 포함

3. 출력 형식 강제
   - JSON 형식 요청 시 "다른 텍스트 없이 JSON만" 강조
   - 후처리로 형식 검증 및 정제

4. Temperature 조절
   - 일관성 필요: 0.3-0.5 (토픽 분류, 피드백)
   - 창의성 필요: 0.7-0.9 (대화 생성)

5. 모델 선택 전략 (HyperCLOVA X 기준)
   - SEED 14B Think: 복잡한 추론, 가이드 생성
   - Chat 모델: 실시간 대화, 간단한 피드백

═══════════════════════════════════════════════════════════════════
개선 로드맵
═══════════════════════════════════════════════════════════════════

Phase 1: 기본 안정화 (1-2주)
  └─ 프롬프트 구조화, Few-shot 예시 추가
  └─ 후처리 로직 구현 (형식 검증)
  └─ 기본 테스트 케이스 구축

Phase 2: 성능 최적화 (2-3주)
  └─ Temperature, max_tokens 튜닝
  └─ 프롬프트 A/B 테스트
  └─ 에러 케이스 분석 및 프롬프트 보완

Phase 3: 고급 기능 (3-4주)
  └─ 멀티턴 컨텍스트 관리
  └─ 사용자 피드백 기반 프롬프트 개선
  └─ HyperCLOVA X 특화 최적화
""")

╔══════════════════════════════════════════════════════════════════╗
║  LLM 프롬프트 종합 성능 분석                                  ║
╚══════════════════════════════════════════════════════════════════╝

═══════════════════════════════════════════════════════════════════
기능별 성능 요약
═══════════════════════════════════════════════════════════════════

┌─────────────────────┬──────────┬──────────┬─────────────────────┐
│ 기능                │ 달성도   │ 신뢰도   │ 주요 과제           │
├─────────────────────┼──────────┼──────────┼─────────────────────┤
│ 아바타 대화 생성    │ 92%      │ 높음     │ 말투 일관성         │
│ 실시간 피드백       │ 96%      │ 높음     │ 설명 수준 조절      │
│ 토픽 라벨링         │ 90%      │ 중간     │ 형식 오류 처리      │
│ 대화 가이드         │ 95%      │ 높음     │ 문화 맥락 깊이      │
│ 표현 추천           │ 88%      │ 중간     │ 자연스러움 향상     │
└─────────────────────┴──────────┴──────────┴─────────────────────┘

═══════════════════════════════════════════════════════════════════
공통 개선 전략
═══════════════════════════════════════════════

---
##  Summary

### 문서화된 LLM 기능

| # | 기능 | 프롬프트 | 달성도 |
|---|------|----------|--------|
| 1 | 아바타 대화 생성 | `generate_avatar_system_prompt()` | 92% |
| 2 | 실시간 피드백 | `generate_feedback_prompt()` | 96% |
| 3 | 토픽 라벨링 | `generate_topic_labeling_prompt()` | 90% |
| 4 | 대화 가이드 | `generate_conversation_guide_prompt()` | 95% |

### 각 섹션 포함 내용
-  전체 프롬프트 (System + User)
-  입력 예시
-  출력 예시 (실제 GPT 응답)
-  의도 달성 수준 분석표
- 문제점 및 개선책

---
**Talkativ - LLM 기반 한국어 대화 코칭 시스템**